### __1. Extracting Forecast weather data from API URL__

### 1.1 Daily weather data extraction (Containing sunrise and sunset daily data)

In [11]:
import requests
import csv

# API URL
url = "https://api.open-meteo.com/v1/forecast?latitude=51.4552&longitude=-2.5966&hourly=temperature_2m,relativehumidity_2m,shortwave_radiation&forecast_days=14"

# Send GET request to the API
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    data = response.json()

    # Extract relevant data from the response
    hourly_data = data["hourly"]
    column_names = ["time", "temperature_2m", "relativehumidity_2m", "shortwave_radiation"]

    # Write the data to a CSV file
    with open("weather_forecast_HOURLY.csv", "w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(column_names)
        for i in range(len(hourly_data["time"])):
            row = [hourly_data[column][i] for column in column_names]
            writer.writerow(row)
            
    print("CSV file created successfully.")
else:
    print("Failed to fetch data from the API.")

CSV file created successfully.


### 1.2 Hourly weather data extraction (Containing temperature, humidity and shortwave_radiation hourly data)

In [12]:
import requests
import csv

# Fetch data from the API
url = "https://api.open-meteo.com/v1/forecast?latitude=51.4552&longitude=-2.5966&daily=sunrise,sunset&timezone=Europe%2FLondon&forecast_days=14"
response = requests.get(url)
data = response.json()

# Prepare CSV file
csv_filename = "weather_forecast_DAILY.csv"

# Extract the required data from the JSON
dates = data["daily"]["time"]
sunrises = data["daily"]["sunrise"]
sunsets = data["daily"]["sunset"]

# Write data to CSV
with open(csv_filename, "w", newline="") as csv_file:
    writer = csv.writer(csv_file)

    # Write header
    writer.writerow(["Date", "Sunrise", "Sunset"])

    # Write daily data
    for date, sunrise, sunset in zip(dates, sunrises, sunsets):
        writer.writerow([date, sunrise, sunset])

print("Conversion completed. The data has been saved in", csv_filename)

Conversion completed. The data has been saved in weather_forecast_DAILY.csv


### __2. Forecast Weather data cleaning__

In [13]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [14]:
# Load the datasets
weather_forecast_daily = pd.read_csv("weather_forecast_DAILY.csv")
weather_forecast_hourly = pd.read_csv("weather_forecast_HOURLY.csv")

In [15]:
# Rename the columns in weather_data_hourly
weather_forecast_hourly = weather_forecast_hourly.rename(columns={'temperature_2m': 'Temperature', 'relativehumidity_2m': 'Humidity', "shortwave_radiation": "Shortwave_Radiation"})

In [16]:
# Convert 'time' column in hourly dataset to datetime
weather_forecast_hourly['time'] = pd.to_datetime(weather_forecast_hourly['time'])
weather_forecast_daily['Date'] = pd.to_datetime(weather_forecast_daily['Date'])

### 2.1 Converting the Hourly weather data into Daily weather data calculating the mean of the variables and merged it

In [17]:
# Group hourly data by date and calculate mean
hourly_data_mean = weather_forecast_hourly.groupby(weather_forecast_hourly['time'].dt.date).mean()

# Reset index to make date a column
hourly_data_mean = hourly_data_mean.reset_index()

# Convert 'Date' column in daily data to datetime
weather_forecast_daily['Date'] = pd.to_datetime(weather_forecast_daily['Date'])

# Merge hourly mean data with daily data based on date
merged_data = weather_forecast_daily.merge(hourly_data_mean, left_on=weather_forecast_daily['Date'].dt.date, right_on=hourly_data_mean['time'], how='left')

# Drop unnecessary columns from the merged data
merged_data.drop(['key_0', 'time'], axis=1, inplace=True)

# Calculate hours of sunlight per day
merged_data['Sunrise'] = pd.to_datetime(merged_data['Sunrise'])
merged_data['Sunset'] = pd.to_datetime(merged_data['Sunset'])
merged_data['Sunlight_hours'] = (merged_data['Sunset'] - merged_data['Sunrise']).dt.total_seconds() / 3600

# Print the merged data
print(merged_data)

         Date             Sunrise              Sunset  Temperature   Humidity  \
0  2023-07-06 2023-07-06 05:01:00 2023-07-06 21:28:00    17.220833  60.000000   
1  2023-07-07 2023-07-07 05:02:00 2023-07-07 21:27:00    20.450000  56.583333   
2  2023-07-08 2023-07-08 05:03:00 2023-07-08 21:27:00    19.404167  70.541667   
3  2023-07-09 2023-07-09 05:04:00 2023-07-09 21:26:00    18.029167  73.458333   
4  2023-07-10 2023-07-10 05:05:00 2023-07-10 21:25:00    18.004167  73.541667   
5  2023-07-11 2023-07-11 05:06:00 2023-07-11 21:25:00    17.770833  78.750000   
6  2023-07-12 2023-07-12 05:07:00 2023-07-12 21:24:00    16.295833  78.708333   
7  2023-07-13 2023-07-13 05:08:00 2023-07-13 21:23:00    16.325000  79.416667   
8  2023-07-14 2023-07-14 05:09:00 2023-07-14 21:22:00    15.920833  74.833333   
9  2023-07-15 2023-07-15 05:11:00 2023-07-15 21:21:00    16.716667  72.541667   
10 2023-07-16 2023-07-16 05:12:00 2023-07-16 21:20:00    16.375000  82.416667   
11 2023-07-17 2023-07-17 05:

In [18]:
merged_data

,Date,Sunrise,Sunset,Temperature,Humidity,Shortwave_Radiation,Sunlight_hours
0,2023-07-06,2023-07-06 05:01:00,2023-07-06 21:28:00,17.220833,60.000000,264.250000,16.450000
1,2023-07-07,2023-07-07 05:02:00,2023-07-07 21:27:00,20.450000,56.583333,358.125000,16.416667
2,2023-07-08,2023-07-08 05:03:00,2023-07-08 21:27:00,19.404167,70.541667,209.000000,16.400000
3,2023-07-09,2023-07-09 05:04:00,2023-07-09 21:26:00,18.029167,73.458333,221.166667,16.366667
4,2023-07-10,2023-07-10 05:05:00,2023-07-10 21:25:00,18.004167,73.541667,217.416667,16.333333
5,2023-07-11,2023-07-11 05:06:00,2023-07-11 21:25:00,17.770833,78.750000,231.916667,16.316667
6,2023-07-12,2023-07-12 05:07:00,2023-07-12 21:24:00,16.295833,78.708333,164.583333,16.283333
7,2023-07-13,2023-07-13 05:08:00,2023-07-13 21:23:00,16.325000,79.416667,216.458333,16.250000
8,2023-07-14,2023-07-14 05:09:00,2023-07-14 21:22:00,15.920833,74.833333,262.541667,16.216667
9,2023-07-15,2023-07-15 05:11:00,2023-07-15 21:21:00,16.716667,72.541667,302.916667,16.166667


In [19]:
# From all the variables of the historic weather data, the most important ones were selected
selected_columns = ["Temperature", "Humidity", "Shortwave_Radiation", "Date", "Sunlight_hours"]
new_weather_df = merged_data[selected_columns]

In [20]:
new_weather_df

,Temperature,Humidity,Shortwave_Radiation,Date,Sunlight_hours
0,17.220833,60.000000,264.250000,2023-07-06,16.450000
1,20.450000,56.583333,358.125000,2023-07-07,16.416667
2,19.404167,70.541667,209.000000,2023-07-08,16.400000
3,18.029167,73.458333,221.166667,2023-07-09,16.366667
4,18.004167,73.541667,217.416667,2023-07-10,16.333333
5,17.770833,78.750000,231.916667,2023-07-11,16.316667
6,16.295833,78.708333,164.583333,2023-07-12,16.283333
7,16.325000,79.416667,216.458333,2023-07-13,16.250000
8,15.920833,74.833333,262.541667,2023-07-14,16.216667
9,16.716667,72.541667,302.916667,2023-07-15,16.166667
